# 최종 가격 예측 모델 (시금치 & 오이)
*시차 특성과 스태킹 앙상블을 적용하여 두 품목의 예측 성능을 모두 최적화합니다.*

### 1. 라이브러리 임포트

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

### 2. 데이터 로드

In [ ]:
cold_df = pd.read_excel('../data/mon_cold.xlsx')
wind_df = pd.read_excel('../data/mon_wind.xlsx')
hot_df = pd.read_excel('../data/mon_hot.xlsx')
price_df = pd.read_excel('../data/region_price.xlsx')
weather_df = pd.read_csv('../data/region_weather.csv')
trade_df = pd.read_excel('../data/spinach_cucumber_df.xlsx')
print('데이터 로드 완료.')

데이터 로드 완료.


### 3. 데이터 전처리 및 병합

In [3]:
cold_df_melted = cold_df.melt(id_vars=['지역'], var_name='날짜', value_name='한파발생')
wind_df_melted = wind_df.melt(id_vars=['지역'], var_name='날짜', value_name='태풍발생')
hot_df_melted = hot_df.melt(id_vars=['지역'], var_name='날짜', value_name='폭염발생')

for df in [price_df, weather_df, trade_df]:
    df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce').dt.strftime('%Y-%m')

merged_df = pd.merge(price_df, weather_df, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, cold_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, wind_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, hot_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, trade_df, on=['품목', '날짜'], how='left')

print('데이터 병합 완료.')

데이터 병합 완료.


### 4. 특성 공학 (시차, 시간 변수)

In [4]:
merged_df['날짜'] = pd.to_datetime(merged_df['날짜'])
merged_df['연도'] = merged_df['날짜'].dt.year
merged_df['월'] = merged_df['날짜'].dt.month

merged_df.sort_values(by=['지역', '품목', '날짜'], inplace=True)

merged_df['가격_시차1'] = merged_df.groupby(['지역', '품목'])['평균가격'].shift(1)

print('시차 및 시간 특성 생성 완료.')

시차 및 시간 특성 생성 완료.


In [13]:
merged_df['가격_시차1'].describe()

count      8767.000000
mean      18613.527182
std       12823.793336
min           0.000000
25%        9120.904062
50%       15410.539139
75%       24757.844756
max      124917.142857
Name: 가격_시차1, dtype: float64

### 5. 결측치 처리 및 최종 데이터셋 정제

In [30]:
merged_df.dropna(subset=['평균가격', '가격_시차1'], inplace=True)
merged_df = merged_df[merged_df['평균가격'] > 0].copy()

merged_df.fillna(0, inplace=True)

print('결측치 처리 완료.')

결측치 처리 완료.


### 6. 원-핫 인코딩

In [31]:
final_df = pd.get_dummies(merged_df, columns=['지역'], prefix='지역', drop_first=True)

print('원-핫 인코딩 적용 완료.')

final_df.info()

원-핫 인코딩 적용 완료.
<class 'pandas.core.frame.DataFrame'>
Index: 8765 entries, 163 to 8969
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   날짜                  8765 non-null   datetime64[ns]
 1   품목                  8765 non-null   object        
 2   평균가격                8765 non-null   float64       
 3   총거래물량               8765 non-null   float64       
 4   평균기온(°C)            8765 non-null   float64       
 5   월합강수량(00~24h만)(mm)  8765 non-null   float64       
 6   평균풍속(m/s)           8765 non-null   float64       
 7   최심적설(cm)            8765 non-null   float64       
 8   한파발생                8765 non-null   float64       
 9   태풍발생                8765 non-null   float64       
 10  폭염발생                8765 non-null   float64       
 11  수입중량                8765 non-null   float64       
 12  수출중량                8765 non-null   float64       
 13  연도                  8765 non-null   

### 7. 모델 학습 및 평가 함수 정의

In [32]:
def train_optimized_model(df, vegetable_name):
    target_df = df[df['품목'] == vegetable_name].copy()
    base_features = ['평균기온(°C)', '월합강수량(00~24h만)(mm)', '평균풍속(m/s)', '최심적설(cm)', '한파발생', '태풍발생', '폭염발생', '연도', '월', '수출중량', '수입중량', '가격_시차1']
    region_features = [col for col in target_df.columns if col.startswith('지역_')]
    features = base_features + region_features
    
    target = '평균가격'
    
    X_train, X_test, y_train, y_test = train_test_split(target_df[features], target_df[target], test_size=0.2, random_state=42)

    estimators = [('rf', RandomForestRegressor(random_state=42)), ('xgb', XGBRegressor(random_state=42)), ('lgbm', LGBMRegressor(random_state=42, verbose=-1))]
    stacking_reg = StackingRegressor(estimators=estimators, final_estimator=Ridge(alpha=1.0), cv=5)

    params = {'rf__n_estimators': [100, 200], 'xgb__n_estimators': [100, 200], 'lgbm__n_estimators': [100, 200]}

    print(f'--- {vegetable_name} 최종 최적화 모델 튜닝 시작 ---')
    grid_search = GridSearchCV(estimator=stacking_reg, param_grid=params, cv=3, n_jobs=-1, scoring='r2', verbose=2)
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f'--- {vegetable_name} 최종 최적화 모델 평가 결과 ---')
    print(f'MAE: {mae:.4f}')
    print(f'MSE: {mse:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'R-squared: {r2:.4f}')
    print(f'Best Hyperparameters: {grid_search.best_params_}')
    print('-'*80)

### 8. 모델 학습 실행

In [33]:
train_optimized_model(final_df, '시금치')
train_optimized_model(final_df, '오이')

--- 시금치 최종 최적화 모델 튜닝 시작 ---
Fitting 3 folds for each of 8 candidates, totalling 24 fits
--- 시금치 최종 최적화 모델 평가 결과 ---
MAE: 1940.9941
MSE: 9953642.8815
RMSE: 3154.9394
R-squared: 0.9178
Best Hyperparameters: {'lgbm__n_estimators': 200, 'rf__n_estimators': 200, 'xgb__n_estimators': 100}
--------------------------------------------------------------------------------
--- 오이 최종 최적화 모델 튜닝 시작 ---
Fitting 3 folds for each of 8 candidates, totalling 24 fits
--- 오이 최종 최적화 모델 평가 결과 ---
MAE: 2071.1861
MSE: 8507423.1140
RMSE: 2916.7487
R-squared: 0.9017
Best Hyperparameters: {'lgbm__n_estimators': 100, 'rf__n_estimators': 200, 'xgb__n_estimators': 200}
--------------------------------------------------------------------------------
